#property indicator_chart_window
#property indicator_buffers 1
#property indicator_plots   1

#property indicator_label1  "Tick Developing POC"
#property indicator_type1   DRAW_LINE
#property indicator_color1  clrDeepSkyBlue
#property indicator_width1  2

// ─────────────────────────────────────
input double PriceStepMult = 1.0;

double PocLine[];

// --- tick profile state
double step;
double pLow, pHigh;
double currentPOC = EMPTY_VALUE;

double volMap[];
int    mapSize = 0;
bool   initialized = false;

// ─────────────────────────────────────
int OnInit()
{
   SetIndexBuffer(0, PocLine, INDICATOR_DATA);
   ArraySetAsSeries(PocLine, true);

   step = _Point * PriceStepMult;
   return INIT_SUCCEEDED;
}

// ─────────────────────────────────────
void OnTick()
{
   double price = SymbolInfoDouble(_Symbol, SYMBOL_BID);

   if(!initialized)
   {
      pLow = pHigh = price;
      mapSize = 1;
      ArrayResize(volMap, 1);
      volMap[0] = 1.0;
      currentPOC = price;
      initialized = true;
      return;
   }

   // --- expand map if needed
   if(price < pLow)
   {
      int add = int((pLow - price) / step) + 1;
      int old = mapSize;
      mapSize += add;

      ArrayResize(volMap, mapSize);
      for(int i = old; i < mapSize; i++)
         volMap[i] = 0.0;

      pLow -= add * step;
   }
   else if(price > pHigh)
   {
      int add = int((price - pHigh) / step) + 1;
      int old = mapSize;
      mapSize += add;

      ArrayResize(volMap, mapSize);
      for(int i = old; i < mapSize; i++)
         volMap[i] = 0.0;

      pHigh += add * step;
   }

   int idx = int((price - pLow) / step);
   if(idx >= 0 && idx < mapSize)
      volMap[idx] += 1.0;

   // --- recompute POC
   double maxVol = 0.0;
   int pocIdx = 0;

   for(int i = 0; i < mapSize; i++)
   {
      if(volMap[i] > maxVol)
      {
         maxVol = volMap[i];
         pocIdx = i;
      }
   }

   currentPOC = pLow + pocIdx * step;
}

// ─────────────────────────────────────
int OnCalculate(
   const int rates_total,
   const int prev_calculated,
   const datetime& time[],
   const double& open[],
   const double& high[],
   const double& low[],
   const double& close[],
   const long& tick_volume[],
   const long& volume[],
   const int& spread[]
)
{
   if(!initialized || rates_total < 2)
      return prev_calculated;

   // --- shift previous value forward
   if(prev_calculated > 0)
      PocLine[1] = PocLine[0];

   // --- write developing POC only on current bar
   PocLine[0] = currentPOC;

   return rates_total;
}
